In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer , Embedding , LSTM , TimeDistributed , Dense , Activation
import glob

---

# Data Downloading

In [38]:
!gdown --id 12hcOY1qoyCzfN5Gr4cNYHa_fun8h88fL

Downloading...
From: https://drive.google.com/uc?id=12hcOY1qoyCzfN5Gr4cNYHa_fun8h88fL
To: /content/LST20-21.zip
100% 345M/345M [00:02<00:00, 118MB/s]


In [39]:
!unzip /content/LST20-21.zip -d /content/Datasets/

Archive:  /content/LST20-21.zip
   creating: /content/Datasets/LST20-21/
   creating: /content/Datasets/LST20-21/Annotation Guidelines/
  inflating: /content/Datasets/LST20-21/Annotation Guidelines/LST20 Annotation Guideline.pdf  
  inflating: /content/Datasets/LST20-21/Annotation Guidelines/LST20 Brief Specification.pdf  
  inflating: /content/Datasets/LST20-21/LST20.zip  
  inflating: /content/Datasets/LST20-21/LST21.zip  


In [40]:
!unzip /content/Datasets/LST20-21/LST20.zip  -d /content/Datasets/LST20/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Datasets/LST20/LST20/test/._T12755.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12756.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12757.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12759.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12761.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12762.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12763.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12764.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12765.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12766.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12767.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12768.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12770.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12771.txt  
  inflating: /content/Datasets/LST20/LST20/test/._T12772.txt  
  in

In [41]:
!gdown --id 1LNdjbQ4TqP1cMWM0vNCfIC5lYJFiz58e

Downloading...
From: https://drive.google.com/uc?id=1LNdjbQ4TqP1cMWM0vNCfIC5lYJFiz58e
To: /content/training.csv
100% 87.4M/87.4M [00:00<00:00, 286MB/s]


# Encoding and Decoding Stuff

- At this place we are going to do the encoding and decoding stuff

In [42]:
data_df = pd.read_csv("training.csv")
training_df = data_df[["text","POS"]].drop_duplicates(['text']).reset_index()

In [43]:
all_vocab = np.concatenate([training_df["text"].unique(),training_df["POS"].unique() , np.array(['_'])])
len(all_vocab)

48905

In [44]:
idx = 0
encoder = {}
for i in all_vocab :
  encoder[i] = idx
  idx+=1
decoder = {i : v for v,i in encoder.items()}

In [75]:
def encode(text) :
  if text in encoder :
    return encoder[text]
  return 48906
def decode(text) :
  if text in decoder : 
    return decoder[text]
  return "_"
def encode_label(text) :
  payload = {'AJ' : 0, 'AV' : 1, 'AX' :2, 'CC' : 3, 'CL' : 4, 'FX' : 5, 'IJ' : 6, 'NG' : 7, 'NN' : 8, 'NU' : 9, 'O' : 10, 'PA' : 11, 'PR' : 12, 'PS' :13, 'PU' :14, 'VV' :15, 'XX' : 16}
  return payload[text]
def decode_label(text) :
  payload = { 0 : 'AJ', 1 : 'AV',  2:'AX' ,  3 : 'CC',  4 : 'CL', 5 : 'FX', 6 : 'IJ', 7 : 'NG', 8 : 'NN', 9 : 'NU', 10 : 'O', 11 : 'PA',  12 : 'PR', 13 :'PS',  14:'PU',  15:'VV', 16 : 'XX'}
  return payload[text]

In [46]:
print(encoder)

{'แชมป์': 0, 'สิงห์คลาสสิก': 1, 'กวาด': 2, 'รางวัล': 3, 'แสน': 4, 'สี่': 5, 'หมื่น': 6, 'บาท': 7, 'ธรรมนูญ': 8, '_': 48904, 'ศรีโรจน์': 10, 'เก็บ': 11, 'เพิ่ม': 12, '4': 13, 'อันเดอร์พาร์': 14, '68': 15, 'เข้า': 16, 'ป้าย': 17, 'รับ': 18, 'ใน': 19, 'การ': 20, 'เล่น': 21, 'อาชีพ': 22, '19': 23, 'ปี': 24, 'เป็น': 25, 'ครั้ง': 26, 'ที่': 27, '8': 28, 'ชีวิต': 29, 'ด้วย': 30, 'สกอร์': 31, '18': 32, '270': 33, 'ส่วน': 34, 'สัตยา': 35, 'ทรัพย์อัประไมย': 36, '3': 37, 'ร่วม': 38, 'รอบ': 39, 'สุดท้าย': 40, 'ของ': 41, 'กอล์ฟสิงห์คลาสสิค': 42, 'ชิง': 43, 'เงิน': 44, 'รวม': 45, '1.5': 46, 'ล้าน': 47, 'สนามบางปะกงริเวอร์ไซด์': 48, 'กอล์ฟคลับ': 49, 'พาร์': 50, '72': 51, 'ระยะ': 52, '7,196': 53, 'หลา': 54, 'จ.': 55, 'ฉะเชิงเทรา': 56, 'เมื่อ': 57, 'วันที่': 58, '5': 59, 'ก.ค.': 60, 'ผ่าน': 61, 'มา': 62, 'แข่งขัน': 63, 'โปร': 64, 'จุ๋ม': 65, 'วัย': 66, '40': 67, 'หวด': 68, 'เบอร์ดี้': 69, 'หลุม': 70, ',': 71, '11': 72, '14': 73, '17': 74, 'เสีย': 75, '1': 76, 'โบกี้': 77, '16': 78, 'จบ': 79, 'คว้า': 80

In [47]:
training_df["e_text"] = training_df["text"].apply(lambda x : encode(x))
training_df["e_pos"] = training_df["POS"].apply(lambda x : encode_label(x))

In [48]:
training_df

,index,text,POS,e_text,e_pos
0,0,แชมป์,NN,0,8
1,1,สิงห์คลาสสิก,NN,1,8
2,2,กวาด,VV,2,15
3,3,รางวัล,NN,3,8
4,4,แสน,NU,4,9
...,...,...,...,...,...
48883,2710324,พ.ต.อ,NN,48883,8
48884,2710326,สรายุทธ,NN,48884,8
48885,2710328,สงวนโภคัย,NN,48885,8
48886,2710359,พรุ้ง,NN,48886,8


# Creating the Train Set 

- in this part we are going to create the train set base on the batch size so that we don't have to do padding with our data

In [49]:
words_payload = []
labels_payload = []

for i in glob.glob('/content/Datasets/LST20/LST20/train/*') :
  fp = open(i,"r")
  for line in fp.readlines() :
    if line == '\n' :
      continue
    else :
      data_ = line[:-2].split("\t")
      words_payload.append(data_[0])
      labels_payload.append(data_[1])

In [50]:
words_payload_encoded = [encode(r) for r in words_payload]
labels_payload_encoded = [encode_label(r) for r in labels_payload]

In [51]:
BATCH_SIZE = 32

In [52]:
np_word_encoded = np.array(words_payload_encoded)
np_label_encoded = np.array(labels_payload_encoded)

In [53]:
print(len(np_word_encoded))

2714726


In [54]:
batched_word = np_word_encoded[:-1*(np_word_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)
batched_label = np_label_encoded[:-1*(np_label_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)

In [90]:
print(batched_word[0])

[  186 11786   186   500   130  1152  5962  1644  7534  1212 48904  2499
   470   130  1152  2335  1028   198   186  4205   186 11786   399   386
   191   674  5962  1644  3500  2335  1028   198]


# Create Validation Set

In [56]:
words_payload_valid = []
labels_payload_valid = []

for i in glob.glob('/content/Datasets/LST20/LST20/eval/*') :
  fp = open(i,"r")
  for line in fp.readlines() :
    if line == '\n' :
      continue
    else :
      data_ = line[:-2].split("\t")
      words_payload_valid.append(data_[0])
      labels_payload_valid.append(data_[1])

In [57]:
words_payload_encoded_valid = [encode(r) for r in words_payload_valid]
labels_payload_encoded_valid = [encode_label(r) for r in labels_payload_valid]

In [58]:
BATCH_SIZE = 32
np_word_encoded_valid = np.array(words_payload_encoded_valid)
np_label_encoded_valid = np.array(labels_payload_encoded_valid)

In [59]:
batched_word_valid = np_word_encoded_valid[:-1*(np_word_encoded_valid.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)
batched_label_valid = np_label_encoded_valid[:-1*(np_label_encoded_valid.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)

In [60]:
batched_label_valid

array([[ 8,  8, 14, ..., 15, 14,  8],
       [ 5, 15,  8, ...,  3, 13, 15],
       [ 8,  5, 15, ..., 15,  5, 15],
       ...,
       [15,  0, 14, ...,  0,  1, 14],
       [ 8,  2, 15, ..., 15,  1, 15],
       [13,  8,  8, ..., 14,  8,  3]])

In [61]:
train_dataset = tf.data.Dataset.from_tensor_slices((batched_word, batched_label))
test_dataset = tf.data.Dataset.from_tensor_slices((batched_word_valid, batched_label_valid))

# Model Creation

- In this part I'm going to test of several models

In [67]:
# InputLayer , Embedding , LSTM , TimeDistributed , Dense , Activation
model = Sequential() 
model.add(InputLayer(input_shape=(BATCH_SIZE,)))
model.add(Embedding(len(all_vocab)+2,64))
model.add(LSTM(64,return_sequences=True))
model.add(TimeDistributed(Dense(17)))
model.add(Activation('softmax'))
model.compile(optimizer="Adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'], run_eagerly=True)

In [68]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 32, 64)            3130048   
                                                                 
 lstm_4 (LSTM)               (None, 32, 64)            33024     
                                                                 
 time_distributed_4 (TimeDis  (None, 32, 17)           1105      
 tributed)                                                       
                                                                 
 activation_4 (Activation)   (None, 32, 17)            0         
                                                                 
Total params: 3,164,177
Trainable params: 3,164,177
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.fit(batched_word , batched_label , epochs=10,validation_data=(batched_word_valid,batched_label_valid))

Epoch 1/10
   7/2652 [..............................] - ETA: 59s - loss: 2.8122 - accuracy: 0.2263 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2652/2652 [==============================] - 60s 23ms/step - loss: 0.2978 - accuracy: 0.9090 - val_loss: 0.1678 - val_accuracy: 0.9439
Epoch 2/10
2652/2652 [==============================] - 60s 22ms/step - loss: 0.1324 - accuracy: 0.9521 - val_loss: 0.1633 - val_accuracy: 0.9454
Epoch 3/10
2652/2652 [==============================] - 60s 22ms/step - loss: 0.1222 - accuracy: 0.9552 - val_loss: 0.1625 - val_accuracy: 0.9455
Epoch 4/10
 453/2652 [====>.........................] - ETA: 46s - loss: 0.1132 - accuracy: 0.9578

KeyboardInterrupt: ignored

In [88]:
z = model.predict(batched_word_valid)
s = np.argmax(z , axis = 2)

In [87]:
func_ = np.vectorize(lambda x : decode_label(x))
s = func_(s)


In [ ]:
func_ = np.vectorize(lambda x : decode(x))

In [86]:
print(s[0])
print(batched_label_valid[0])

[ 8  8 14 14 15 14  8 15  8  8 13  8  5 15  8 14  8  2 15  8  3 15  1  8
  8  8  8  8 14 15 14  8]
[ 8  8 14 14 15 14  8 15  8  8 13  8  5 15  8 14  8  2 15  8  3 15 13  8
  8  8  8  8 14 15 14  8]


#Submission Download

In [91]:
!gdown --id 1cwlaGd-l1yFqT7LctDqxT5ikpJj0vx_g

Downloading...
From: https://drive.google.com/uc?id=1cwlaGd-l1yFqT7LctDqxT5ikpJj0vx_g
To: /content/pos_test.txt.zip
100% 249k/249k [00:00<00:00, 80.8MB/s]


In [92]:
!unzip /content/pos_test.txt.zip -d /content/submission/

Archive:  /content/pos_test.txt.zip
  inflating: /content/submission/pos_test.txt  


In [233]:
fp = open("pos_test.txt","r")
  

In [234]:
word_list_submission = []
for line in fp.readlines() :
  if line != '\n' :
    word_list_submission.append(line[:-1])
  else :
    word_list_submission.append('_')

In [235]:
len(word_list_submission)

103914

In [236]:
word_list_submission = word_list_submission[:-1]

In [237]:
len(word_list_submission)

103913

In [238]:
103913%32

9

In [239]:
word_list_submission += ['_']*23

In [240]:
len(word_list_submission)

103936

In [241]:
print(word_list_submission)

['อย่างไรก็ตาม', 'เครื่อง', 'บิน', 'แอร์บัส', '_', 'เอ', '380', 'ได้', 'ถึง', 'ที่', 'หมาย', 'ที่', 'ท่าอากาศยาน', 'เชียงใหม่', 'ใน', 'เวลา', '_', '14.50', '_', 'น.', 'โดย', 'ใช้', 'เวลา', 'บิน', 'ประมาณ', '1', 'ชั่วโมง', '_', 'มูลนิธิสืบฯ', 'พอใจ', 'ฟื้น', 'ป่าห้วยขาแข้ง', 'มี', 'เสือโคร่ง', 'มาก', 'อันดับ', '2', 'โลก', '_', '1', '_', 'กันยายน', '_', '2550', '17.31', 'น.', '_', 'ประธาน', 'มูลนิธิ', 'สืบ', 'นาคะเสถียร', 'พอใจ', 'ป่า', 'ห้วยขาแข้ง', 'ฟื้นตัว', 'พบ', 'เสือโคร่ง', '2-3', 'ตัว', 'ต่อ', '100', 'ตาราง', 'กิโลเมตร', 'มาก', 'อันดับ', 'ที่', '_', '2', 'ของ', 'โลก', 'ตั้ง', 'เป้า', '5', 'ปี', 'ขึ้น', 'อันดับ', '1', 'แสดง', 'ถึง', 'ยัง', 'มี', 'สัตว์', 'และ', 'พืช', 'ชนิด', 'อื่น', 'ๆ', 'อุดมสมบูรณ์', '_', 'วอน', 'รัฐ', 'เข้าใจ', 'ปรับ', 'นโยบาย', 'จับตา', 'ค้าน', 'กฎหมาย', 'เปิด', 'ป่า', '_', '(', '1', 'ก.ย.', ')', 'นาง', 'รตยา', '_', 'จันทรเทียร', 'ประธาน', 'มูลนิธิ', 'สืบ', 'นาคะเสถียร', 'ให้', 'สัมภาษณ์', 'ถึง', 'สถานการณ์', 'ป่า', 'และ', 'สัตว์', 'ป่า', 'ใน', 'ป่าห้วยขาแข้ง'

In [242]:
word_list_submission_encoded = [encode(i) for i in word_list_submission]

In [243]:
print(word_list_submission_encoded)

[1291, 2286, 771, 6802, 48904, 5963, 21614, 108, 273, 27, 1818, 27, 4770, 1908, 19, 211, 48904, 23207, 48904, 213, 199, 430, 211, 771, 378, 76, 206, 48904, 48906, 4060, 3429, 48906, 152, 48906, 120, 86, 87, 377, 48904, 76, 48904, 857, 48904, 1715, 7346, 213, 48904, 498, 4136, 2086, 12618, 4060, 1866, 48906, 9373, 657, 48906, 1841, 117, 198, 1068, 3617, 4068, 120, 86, 27, 48904, 87, 41, 377, 357, 3770, 59, 24, 388, 86, 76, 662, 273, 519, 152, 1865, 123, 1868, 2154, 628, 372, 12626, 48904, 1588, 572, 2074, 1038, 1494, 397, 2481, 530, 244, 1866, 48904, 207, 76, 209, 210, 1443, 34713, 48904, 34714, 498, 4136, 2086, 12618, 448, 257, 273, 552, 1866, 123, 1865, 1866, 19, 48906, 55, 5283, 104, 266, 20, 2792, 147, 728, 48904, 1164, 657, 104, 6843, 1866, 48906, 289, 25, 6843, 1866, 414, 939, 148, 3552, 25, 390, 147, 1870, 41, 377, 308, 1028, 41, 385, 433, 24, 1664, 48904, 190, 122, 152, 48906, 1036, 271, 1841, 117, 198, 308, 1068, 3617, 4068, 48904, 697, 45, 152, 48906, 378, 2408, 188, 1068, 117

In [247]:
np_word_list_submission_encoded_batched = np.array(word_list_submission_encoded).reshape(-1,BATCH_SIZE)

In [248]:
np_word_list_submission_encoded_batched

array([[ 1291,  2286,   771, ...,  4060,  3429, 48906],
       [  152, 48906,   120, ...,  3617,  4068,   120],
       [   86,    27, 48904, ...,  1494,   397,  2481],
       ...,
       [  501,   262,   370, ..., 48904,   152,   501],
       [   75,    29,   266, ...,   108,   566,   448],
       [  271,    19,    19, ..., 48904, 48904, 48904]])

## submiss

In [249]:
z = model.predict(np_word_list_submission_encoded_batched)

In [250]:
print(z.shape)

(3248, 32, 17)


In [251]:
s = np.argmax(z,axis=2)

In [252]:
func2_ = np.vectorize(lambda x : decode_label(x))

In [253]:
datum = func2_(s)

In [259]:
datum

array([['CC', 'NN', 'VV', ..., 'VV', 'VV', 'NN'],
       ['VV', 'NN', 'VV', ..., 'CL', 'CL', 'VV'],
       ['NN', 'PS', 'PU', ..., 'NN', 'VV', 'VV'],
       ...,
       ['FX', 'NN', 'NN', ..., 'PU', 'VV', 'FX'],
       ['VV', 'NN', 'PS', ..., 'AX', 'VV', 'CC'],
       ['VV', 'PS', 'PS', ..., 'PU', 'PU', 'PU']], dtype='<U2')

In [263]:
s = datum.flatten()

In [264]:
x = s.tolist()[:-23]
print(len(x))

103913


In [265]:
x

['CC',
 'NN',
 'VV',
 'NN',
 'PU',
 'NN',
 'NU',
 'AX',
 'VV',
 'NN',
 'NN',
 'CC',
 'NN',
 'NN',
 'PS',
 'NN',
 'PU',
 'NU',
 'PU',
 'CL',
 'CC',
 'VV',
 'NN',
 'VV',
 'AJ',
 'NU',
 'CL',
 'PU',
 'NN',
 'VV',
 'VV',
 'NN',
 'VV',
 'NN',
 'VV',
 'NN',
 'NU',
 'NN',
 'PU',
 'NU',
 'PU',
 'NN',
 'PU',
 'NU',
 'NU',
 'CL',
 'PU',
 'NN',
 'NN',
 'VV',
 'NN',
 'VV',
 'NN',
 'NN',
 'VV',
 'VV',
 'NN',
 'NU',
 'NN',
 'PS',
 'NU',
 'CL',
 'CL',
 'VV',
 'NN',
 'PS',
 'PU',
 'NU',
 'PS',
 'NN',
 'VV',
 'NN',
 'NU',
 'CL',
 'AV',
 'NN',
 'NU',
 'VV',
 'PS',
 'AX',
 'VV',
 'NN',
 'CC',
 'NN',
 'NN',
 'AJ',
 'PU',
 'VV',
 'PU',
 'VV',
 'NN',
 'VV',
 'VV',
 'NN',
 'VV',
 'VV',
 'NN',
 'VV',
 'NN',
 'PU',
 'PU',
 'NU',
 'NN',
 'PU',
 'NN',
 'NN',
 'PU',
 'NN',
 'NN',
 'NN',
 'VV',
 'NN',
 'CC',
 'NN',
 'PS',
 'NN',
 'NN',
 'CC',
 'NN',
 'NN',
 'PS',
 'NN',
 'NN',
 'NN',
 'CC',
 'PS',
 'FX',
 'VV',
 'NN',
 'NN',
 'PU',
 'AV',
 'VV',
 'CC',
 'NN',
 'NN',
 'NN',
 'CC',
 'VV',
 'NN',
 'NN',
 'NN',
 'VV',

In [266]:
submission_df = pd.read_csv("/content/pos_sample_submission.csv")

In [267]:
submission_df.Predicted = x

In [268]:
submission_df

,Id,Predicted
0,1,CC
1,2,NN
2,3,VV
3,4,NN
4,5,PU
...,...,...
103908,103909,VV
103909,103910,VV
103910,103911,VV
103911,103912,FX


In [269]:
submission_df.to_csv("submission.csv",index=False)

# Submission

In [89]:
model.save('weight.h5')